In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import TensorBoard

# Check if GPU is available
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Define the classes
classes = ['Airbus-A220', 'Airbus-A300', 'Airbus-A310', 'Airbus-A320',
           'Airbus-A330', 'Airbus-A350', 'Airbus-A380', 'Boeing-B707',
           'Boeing-B727', 'Boeing-B737', 'Boeing-B767', 'Boeing-B777',
           'Boeing-B787', 'EF2000']

# Define the path to the dataset
train_dataset_path = "./train"
test_dataset_path = "./test"



Num GPUs Available:  1


In [2]:
# Function to load and preprocess the dataset
def load_dataset(dataset_path):
    data = []
    labels = []

    for i, aircraft_class in enumerate(classes):
        class_path = os.path.join(dataset_path, aircraft_class)
        for image_name in os.listdir(class_path):
            image_path = os.path.join(class_path, image_name)
            image = cv2.imread(image_path)
            #image = cv2.resize(image, (224, 224))  # Resize images to a common size
            image = cv2.resize(image, (128, 128))
            data.append(image)
            labels.append(i)

    data = np.array(data) / 255.0  # Normalize pixel values to be between 0 and 1
    labels = np.array(labels)

    return data, labels

# Load the training dataset
train_data, train_labels = load_dataset(train_dataset_path)

# Load the testing dataset
test_data, test_labels = load_dataset(test_dataset_path)



In [6]:
print(train_data.shape, train_labels.shape, test_data.shape, test_labels.shape)

(4574, 128, 128, 3) (4574,) (2138, 128, 128, 3) (2138,)


In [4]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, Progbar
from keras import models, layers
from tensorflow.python.keras import backend as K

# Assuming you have loaded your train_data, train_labels, test_data, and test_labels

# Define the CNN model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(len(classes), activation='softmax'))

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# TensorBoard callback
tensorboard_callback = TensorBoard(log_dir="logs", histogram_freq=1)

# Create an ImageDataGenerator for training with data augmentation
batch_size = 5  # Decreased batch size to fit within GPU memory
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Create generators for training and validation data
train_generator = train_datagen.flow(train_data, train_labels, batch_size=batch_size)
validation_data = (test_data, test_labels)

# Train the model with progress bar
epochs = 10
steps_per_epoch = len(train_data) // batch_size
validation_steps = len(test_data) // batch_size

# Use Progbar to display the progress bar
progbar = Progbar(target=steps_per_epoch * epochs)

for epoch in range(epochs):
    print(f'Epoch {epoch + 1}/{epochs}')

    # Training
    model.fit(train_generator, steps_per_epoch=steps_per_epoch, validation_data=validation_data, validation_steps=validation_steps, callbacks=[tensorboard_callback])

    # Display progress bar
    progbar.update(steps_per_epoch)

    # Clear session
    K.clear_session()

# Evaluate the model
test_loss, test_acc = model.evaluate(test_data, test_labels)
print(f'Test accuracy: {test_acc}')

# Save the model
model.save('aircraft_classifier_model_gpu.h5')
model.save('aircraft_classifier_model_gpu.keras')


Epoch 1/10
67/67 [==============================] - 3s 35ms/step - loss: 3.6988 - accuracy: 0.0482
Test accuracy: 0.0481758639216423


In [5]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_data, test_labels)
print(f'Test accuracy: {test_acc}')


67/67 [==============================] - 2s 30ms/step - loss: 3.6988 - accuracy: 0.0482
Test accuracy: 0.0481758639216423


In [9]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

classes = ['Airbus-A220', 'Airbus-A300', 'Airbus-A310', 'Airbus-A320',
          'Airbus-A330', 'Airbus-A350', 'Airbus-A380', 'Boeing-B707',
          'Boeing-B727', 'Boeing-B737', 'Boeing-B767', 'Boeing-B777',
          'Boeing-B787', 'EF2000']

def predict_and_highlight_flight(video_path, model):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    
    # Get video properties
    width = int(cap.get(3))
    height = int(cap.get(4))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # Define the codec and create a VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('./output_v2/output.avi', fourcc, fps, (width, height))
    
    # Loop through each frame in the video
    frame_index = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Preprocess the frame
        resized_frame = cv2.resize(frame, (128, 128))  # Modify the size to 128x128
        normalized_frame = resized_frame / 255.0  # Normalize pixel values
        
        # Expand dimensions to match model input shape
        input_frame = np.expand_dims(normalized_frame, axis=0)
        
        # Make predictions using the model
        predictions = model.predict(input_frame)
        
        # Get the predicted class index
        predicted_class_index = np.argmax(predictions)
        
        # Check if the predicted class corresponds to any flight class
        if predicted_class_index in range(len(classes)):
            # Draw bounding box in green
            color = (0, 255, 0)
        else:
            # Draw bounding box in red
            color = (0, 0, 255)
        
        # Highlight or draw bounding box
        cv2.rectangle(frame, (0, 0), (width, height), color, 2)
        
        # Get the predicted class label
        predicted_class_label = classes[predicted_class_index] if predicted_class_index in range(len(classes)) else 'Unknown'
        
        # Display the predicted class label
        cv2.putText(frame, f"Class: {predicted_class_label}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Write the frame to the output video
        out.write(frame)
        
        # Calculate time in seconds
        time_in_seconds = frame_index / fps
        
        # Display the time in seconds
        print(f"Time: {time_in_seconds:.2f} seconds, Predicted Class: {predicted_class_label}")
        
        # Increment the frame index
        frame_index += 1
        
        # Display the frame
        cv2.imshow('Video', frame)
        
        # Break the loop if 'q' key is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Release video capture and writer objects
    cap.release()
    out.release()
    
    # Close all OpenCV windows
    cv2.destroyAllWindows()

# Load the trained model
model = load_model('aircraft_classifier_model_gpu.h5')

# Call the prediction function with the video path and trained model
predict_and_highlight_flight('cat.mp4', model)


1/1 [==============================] - 0s 135ms/step
Time: 0.00 seconds, Predicted Class: EF2000
1/1 [==============================] - 0s 41ms/step
Time: 0.04 seconds, Predicted Class: EF2000
1/1 [==============================] - 0s 33ms/step
Time: 0.08 seconds, Predicted Class: EF2000
1/1 [==============================] - 0s 42ms/step
Time: 0.12 seconds, Predicted Class: EF2000
1/1 [==============================] - 0s 48ms/step
Time: 0.16 seconds, Predicted Class: EF2000
1/1 [==============================] - 0s 32ms/step
Time: 0.20 seconds, Predicted Class: EF2000
1/1 [==============================] - 0s 35ms/step
Time: 0.24 seconds, Predicted Class: EF2000
1/1 [==============================] - 0s 40ms/step
Time: 0.28 seconds, Predicted Class: EF2000
1/1 [==============================] - 0s 39ms/step
Time: 0.32 seconds, Predicted Class: EF2000
1/1 [==============================] - 0s 42ms/step
Time: 0.36 seconds, Predicted Class: EF2000
1/1 [==============================] - 

In [13]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3879218681394507968
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3655335936
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14434595375747836136
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]
